In [4]:
import pandas as pd
import numpy as np
import re

### Movies metadata

In [5]:
movies_path = 'dialogue_data/movie_titles_metadata.tsv'
movies_df = pd.read_csv(movies_path, sep='\t', header=None)
movies_df.rename(columns={0:'ID', 1:'Title', 2:'Year', 3:'IMDB Rating', 4:'Votes', 5:'Genres'}, inplace=True)

In [6]:
movies_df

,ID,Title,Year,IMDB Rating,Votes,Genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']
5,m5,the fifth element,1997,7.5,133756.0,['action' 'adventure' 'romance' 'sci-fi' 'thri...
6,m6,8mm,1999,6.3,48212.0,['crime' 'mystery' 'thriller']
7,m7,a nightmare on elm street 4: the dream master,1988,5.2,13590.0,['fantasy' 'horror' 'thriller']
8,m8,a nightmare on elm street: the dream child,1989,4.7,11092.0,['fantasy' 'horror' 'thriller']
9,m9,the atomic submarine,1959,4.9,513.0,['sci-fi' 'thriller']


### Exporting movie lines dataset as CSV

In [5]:
# line_ids = []
# char_ids = []
# movie_ids = []
# chars = []
# lines = []
# dialogue_path = 'dialogue_data/movie_lines.tsv'
# with open(dialogue_path) as f:
#     for curr in f:
#         L = curr.index('L')
#         U = curr.index('u')
#         M = curr.index('m')
#         C = curr.index('\t', M+1)
#         S = curr.index('\t', C+1)
#         line_id = curr[L:U].strip()
#         char_id = curr[U:M].strip()
#         movie_id = curr[M:C].strip()
#         char = curr[C:S].strip()
#         line = curr[S:].strip()

#         line_ids.append(line_id)
#         char_ids.append(char_id)
#         movie_ids.append(movie_id)
#         chars.append(char)
#         lines.append(line)
#         if len(line_ids)%1000==0:
#             print(len(line_ids))

# lines_df = lines_df.from_dict({'Line ID':line_ids, 'Character ID':char_ids, 'Movie ID':movie_ids,'Character':chars, 'Line':lines})


In [6]:
# lines_df.to_csv('movie_lines_cleaned.csv')

NameError: name 'lines_df' is not defined

### Movies character metadata

In [7]:
lines_df = pd.read_csv('data_cleaned_merged/movie_lines_cleaned.csv')

In [8]:
char_path = 'dialogue_data/movie_characters_metadata.tsv'
char_df = pd.read_csv(char_path, sep='\t', header=None)
char_df.rename(columns={0:'Character ID', 1:'Character', 2:'Movie ID', 3:'Title', 4:'Gender', 5:'Position in Credits'}, inplace=True)
char_df.Gender = char_df.Gender.str.lower()

In [9]:
char_df.head()

,Character ID,Character,Movie ID,Title,Gender,Position in Credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


### Merging character metadata w/ lines data

In [10]:
merged = lines_df.merge(char_df, on=['Character ID','Movie ID'], how='inner', suffixes=['1','2'])
merged.Gender=merged.Gender.str.lower()

In [11]:
merged = merged.dropna()

In [12]:
merged.to_csv('data_cleaned_merged/movie_lines_with_characters.csv')

### Female vs. Male Presence in Credits & Lines

In [13]:
print('Female lines: ' + str(len(merged[merged.Gender =='f'])))
print('Male lines: ' + str(len(merged[merged.Gender =='m'])))
print('Unk gender lines: ' + str(len(merged[merged.Gender =='?'])))

Female lines: 71134
Male lines: 170200
Unk gender lines: 62480


In [14]:
for i in range(1,11):
    filter = char_df["Position in Credits"]==str(i)
    pos = char_df.where(filter, inplace=False).dropna()
    print('#' + str(i) + ' Male Characters: ' + str(len(pos[pos.Gender == 'm'])))
    print('#' + str(i) + ' Female Characters: ' + str(len(pos[pos.Gender == 'f'])))
    print('#' + str(i) + ' Unk Characters: ' + str(len(pos[pos.Gender == '?']) + len(pos[pos.Gender == np.NaN] )))

#1 Male Characters: 408
#1 Female Characters: 89
#1 Unk Characters: 0
#2 Male Characters: 232
#2 Female Characters: 211
#2 Unk Characters: 0
#3 Male Characters: 232
#3 Female Characters: 120
#3 Unk Characters: 0
#4 Male Characters: 185
#4 Female Characters: 83
#4 Unk Characters: 0
#5 Male Characters: 159
#5 Female Characters: 52
#5 Unk Characters: 0
#6 Male Characters: 123
#6 Female Characters: 46
#6 Unk Characters: 0
#7 Male Characters: 86
#7 Female Characters: 39
#7 Unk Characters: 0
#8 Male Characters: 65
#8 Female Characters: 35
#8 Unk Characters: 0
#9 Male Characters: 51
#9 Female Characters: 28
#9 Unk Characters: 0
#10 Male Characters: 32
#10 Female Characters: 22
#10 Unk Characters: 0


In [15]:
merged['Length'] = merged['Line'].apply(lambda d: d.split())

In [17]:
# Total length of lines by gender
merged.groupby('Gender').sum()

,Unnamed: 0
Gender,
?,9582114327
f,10998298267
m,25663706555


In [18]:
# Avg length of lines by gender
merged.groupby('Gender').mean()

,Unnamed: 0
Gender,
?,153362.905362
f,154613.803062
m,150785.584929


In [28]:
merged_all = movies_df.merge(merged, on=['Title'], how='inner')

In [30]:
merged_all.to_csv('data_cleaned_merged/movie_lines_with_all_metadata.csv')

In [73]:
male_line_count = male_line_count[['Line']]
female_line_count = female_line_count[['Line']]
male_line_count.rename(columns={'Line':'Male_Lines'}, inplace=True)
female_line_count.rename(columns={'Line':'Female_Lines'}, inplace=True)

In [75]:
line_pct_by_gender = female_line_count.merge(male_line_count, on='Movie ID')

In [78]:
line_pct_by_gender['Total_Lines'] = line_pct_by_gender['Female_Lines'] + line_pct_by_gender['Male_Lines']
line_pct_by_gender['Female_Percent'] = line_pct_by_gender['Female_Lines']/line_pct_by_gender['Total_Lines'] 
line_pct_by_gender['Male_Percent'] = line_pct_by_gender['Male_Lines']/line_pct_by_gender['Total_Lines'] 


In [79]:
line_pct_by_gender

,Female_Lines,Male_Lines,Total_Lines,Female_Percent,Male_Percent
Movie ID,,,,,
m0,277,359,636,0.435535,0.564465
m10,104,441,545,0.190826,0.809174
m100,149,775,924,0.161255,0.838745
m101,42,452,494,0.085020,0.914980
m102,233,353,586,0.397611,0.602389
m103,360,362,722,0.498615,0.501385
m104,77,500,577,0.133449,0.866551
m105,423,761,1184,0.357264,0.642736
m106,146,450,596,0.244966,0.755034
